# Cleaning Garmin data

In [275]:
import pandas as pd
import numpy as np
df = pd.read_csv("./garmin_data/garmin.70975009 2.csv")  
df.head()


,ts,dte_tme,rsp_id,data_type,val
0,1.651511e+09,2022-05-02 09:56:58,1,hr,0
1,1.651511e+09,2022-05-02 09:56:58,1,steps,0
2,1.651511e+09,2022-05-02 09:56:58,1,calories,0
3,1.651511e+09,2022-05-02 09:56:58,1,floorsClimbed,0
4,1.651511e+09,2022-05-02 09:56:59,1,intensityMinutes,0


In [276]:
df['dte_tme']

0        2022-05-02 09:56:58
1        2022-05-02 09:56:58
2        2022-05-02 09:56:58
3        2022-05-02 09:56:58
4        2022-05-02 09:56:59
                ...         
62185    2022-08-15 16:36:16
62186    2022-08-15 16:36:16
62187    2022-08-15 16:36:16
62188    2022-08-15 16:36:17
62189    2022-08-15 16:36:17
Name: dte_tme, Length: 62190, dtype: object

In [277]:
df['data_type'].value_counts()

ibi                 33953
hr                  20688
stress               5658
pulseOx               831
calories              547
steps                 357
floorsClimbed          78
intensityMinutes       78
Name: data_type, dtype: int64

## Converting date time data to date for easy preprocessing

In [278]:
df['dte_tme'] = pd.to_datetime(df['dte_tme'])

In [279]:
df['new_date'] = [d.date() for d in df['dte_tme']]

In [280]:
df

,ts,dte_tme,rsp_id,data_type,val,new_date
0,1.651511e+09,2022-05-02 09:56:58,1,hr,0,2022-05-02
1,1.651511e+09,2022-05-02 09:56:58,1,steps,0,2022-05-02
2,1.651511e+09,2022-05-02 09:56:58,1,calories,0,2022-05-02
3,1.651511e+09,2022-05-02 09:56:58,1,floorsClimbed,0,2022-05-02
4,1.651511e+09,2022-05-02 09:56:59,1,intensityMinutes,0,2022-05-02
...,...,...,...,...,...,...
62185,1.660607e+09,2022-08-15 16:36:16,32716,hr,61,2022-08-15
62186,1.660607e+09,2022-08-15 16:36:16,32716,stress,6,2022-08-15
62187,1.660607e+09,2022-08-15 16:36:16,32716,ibi,1049,2022-08-15
62188,1.660607e+09,2022-08-15 16:36:17,32716,hr,60,2022-08-15


### For data types calories, steps, floorsClimbed and intensity in Minutes we need the last value(max value) for each date as it has a increasing trend. For data type hr we replaced 0 values with resting heart rate(random number btw 60-100). To do this cleaning we divided into 2 parts.

### Part1 - dealing with max values

In [281]:
df_part1 = df.groupby(['new_date','data_type'], as_index=False)['val'].max()
df_part1

,new_date,data_type,val
0,2022-05-02,calories,803
1,2022-05-02,floorsClimbed,0
2,2022-05-02,hr,0
3,2022-05-02,ibi,472
4,2022-05-02,intensityMinutes,0
...,...,...,...
320,2022-08-15,ibi,1049
321,2022-08-15,intensityMinutes,0
322,2022-08-15,pulseOx,0
323,2022-08-15,steps,885


In [282]:
df1 = df_part1[(df_part1['data_type']=='calories') | (df_part1['data_type']=='steps') | (df_part1['data_type']=='floorsClimbed') | (df_part1['data_type']=='intensityMinutes')]
df1

,new_date,data_type,val
0,2022-05-02,calories,803
1,2022-05-02,floorsClimbed,0
4,2022-05-02,intensityMinutes,0
6,2022-05-02,steps,0
7,2022-05-09,calories,899
...,...,...,...
315,2022-08-12,steps,1203
317,2022-08-15,calories,884
318,2022-08-15,floorsClimbed,0
321,2022-08-15,intensityMinutes,0


### Part 2 - dealing with hr attribute

In [283]:
condition  = (df['val'] == 0 )
df.loc[condition, 'val'] = np.random.randint(low=60, high=100)
df

,ts,dte_tme,rsp_id,data_type,val,new_date
0,1.651511e+09,2022-05-02 09:56:58,1,hr,97,2022-05-02
1,1.651511e+09,2022-05-02 09:56:58,1,steps,97,2022-05-02
2,1.651511e+09,2022-05-02 09:56:58,1,calories,97,2022-05-02
3,1.651511e+09,2022-05-02 09:56:58,1,floorsClimbed,97,2022-05-02
4,1.651511e+09,2022-05-02 09:56:59,1,intensityMinutes,97,2022-05-02
...,...,...,...,...,...,...
62185,1.660607e+09,2022-08-15 16:36:16,32716,hr,61,2022-08-15
62186,1.660607e+09,2022-08-15 16:36:16,32716,stress,6,2022-08-15
62187,1.660607e+09,2022-08-15 16:36:16,32716,ibi,1049,2022-08-15
62188,1.660607e+09,2022-08-15 16:36:17,32716,hr,60,2022-08-15


In [284]:
df_part2= df.groupby(['new_date','data_type'], as_index=False)['val'].mean()

In [285]:
df2 = df_part2[(df_part2['data_type']=='hr')]
df2

,new_date,data_type,val
2,2022-05-02,hr,97.000000
9,2022-05-09,hr,74.707979
17,2022-05-10,hr,66.941176
25,2022-05-11,hr,70.890977
33,2022-05-12,hr,75.716216
41,2022-05-13,hr,80.809425
49,2022-05-16,hr,73.066667
57,2022-05-17,hr,77.887850
64,2022-05-18,hr,70.347267
70,2022-05-19,hr,86.166667


### Merging 2 parts

In [286]:
frames = [df1, df2]

result = pd.concat(frames)
result

,new_date,data_type,val
0,2022-05-02,calories,803.000000
1,2022-05-02,floorsClimbed,0.000000
4,2022-05-02,intensityMinutes,0.000000
6,2022-05-02,steps,0.000000
7,2022-05-09,calories,899.000000
...,...,...,...
287,2022-08-03,hr,65.112335
295,2022-08-05,hr,63.993722
303,2022-08-10,hr,74.372881
311,2022-08-12,hr,77.810185


In [287]:
result = result.sort_values(by=['new_date'])

In [288]:
result

,new_date,data_type,val
0,2022-05-02,calories,803.000000
6,2022-05-02,steps,0.000000
2,2022-05-02,hr,97.000000
1,2022-05-02,floorsClimbed,0.000000
4,2022-05-02,intensityMinutes,0.000000
...,...,...,...
323,2022-08-15,steps,885.000000
317,2022-08-15,calories,884.000000
318,2022-08-15,floorsClimbed,0.000000
321,2022-08-15,intensityMinutes,0.000000


In [289]:
list_dates = result.new_date.unique()


### Transforming grouped row data to column data for analysis

In [290]:
result_col = result.pivot(index = 'new_date', columns='data_type', values='val')
result_col.head

<bound method NDFrame.head of data_type   calories  floorsClimbed         hr  intensityMinutes   steps
new_date                                                                
2022-05-02     803.0            0.0  97.000000               0.0     0.0
2022-05-09     899.0            3.0  74.707979               0.0  1529.0
2022-05-10     905.0            1.0  66.941176               0.0   849.0
2022-05-11     961.0            1.0  70.890977               0.0  1599.0
2022-05-12     919.0            0.0  75.716216               0.0  2005.0
2022-05-13    1707.0            0.0  80.809425               0.0  8488.0
2022-05-16     797.0            0.0  73.066667               0.0   928.0
2022-05-17     874.0            0.0  77.887850               0.0  2063.0
2022-05-18     811.0            NaN  70.347267               NaN     NaN
2022-05-19     993.0            0.0  86.166667               0.0  2077.0
2022-05-20     695.0            0.0  70.347339               0.0   211.0
2022-05-23     824.0 

### Adding participant Id

In [291]:
result_col.insert(0, "Participant Id", 70975009)
result_col.insert(1, "Date", list_dates)

### Saving into csv file

In [292]:
file_name = "final_garmin_data_1.csv"
result_col.to_csv(file_name, encoding='utf-8', index=False)